In [1]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import csv
import pickle
import json
import geopandas as gpd
from branca.colormap import linear

import folium
from folium.plugins import TimeSliderChoropleth

from ipywidgets import widgets, interact, Label, HBox
from ipyleaflet import Map, basemaps, GeoJSON, DrawControl, LayersControl, basemap_to_tiles, Rectangle
import plotly.plotly as py
import plotly.graph_objs as go

from branca.element import Figure

In [2]:
'''pip install -e git://github.com/python-visualization/folium.git@master#egg=folium'''

In [3]:
print(folium.__version__)

In [4]:
pd.__version__

##### work only with may june 2016 data

In [2]:
verify_file = "H:/Yandex machine learning/finall course coursera/verif_bin_stat_5_6_2016.pkl"
predict_file = "H:/Yandex machine learning/finall course coursera/3) Taxi time serie prediction/week 6/final_taxi_prediction_2015-05-06.csv"

In [3]:
NY_long_west = -74.25559
NY_long_east = -73.70001
NY_latt_south = 40.49612
NY_latt_north = 40.91553

start_pred = '2016-06-01T00:00:00.000000000'
end_pred = '2016-06-07T23:00:00.000000000'

In [4]:
with open(verify_file, "rb") as f:
    verify_2month = pickle.load(f)  
    
with open(predict_file, "rb") as f:
    data_predict = pd.read_csv(f, sep="\t")    

In [5]:
print(verify_2month.shape)
print(data_predict.shape)

In [6]:
time = pd.date_range('2016 May 1 00:00:00', '2016 June 30 23:00:00', freq = 'h')

verify_taxi = pd.DataFrame(verify_2month.T[1:, :], index = time, columns=list(map(int, verify_2month.T[0, :])))
data_predict = pd.DataFrame(data_predict.iloc[:, 1:].values, index = time, columns=list(map(int, data_predict.columns[1:])))

In [10]:
verify_taxi.head()

In [19]:
data_predict.head()

##### NY city coordinate grid

In [7]:
lat_grid = np.linspace(NY_latt_south, NY_latt_north, 51)
long_grid = np.linspace(NY_long_west, NY_long_east, 51)

lat, lon = np.meshgrid(lat_grid, long_grid, indexing="ij")
NY_coord_grid = np.dstack((lat, lon))
NY_coord_grid.shape

In [11]:
#nonempty_col = np.ravel(list(verify_taxi.columns.values))
#mtx = np.array([np.zeros(168) if region not in nonempty_col else np.ravel(verify_taxi.loc[start_pred:end_pred][region].values)
#                              for region in range(2500)]).T 
#mtx_ = mtx.reshape(-1, 50, 50)
#mtx_ = mtx_.transpose(0, 2, 1)

'''YlOrBr_05 BuGn_09 Blues_08 PuRd_09 Oranges_09'''
#from branca.colormap import linear
#colormap = linear.YlOrBr_09.scale(verify_taxi.values.min(), verify_taxi.values.max())

### Create interactive maping in time to compare real and predicted taxi calls. 

##### 1) First version

##### create coordinate grid of city regions

In [8]:
ny_json = {
            "type": "FeatureCollection",
            "features": []
          }   

In [9]:
def create_reg_grid(json_d):
    for i in range(50):
        for j in range(50):
            if (50*i + j) in verify_taxi.columns.values: 
                json_d["features"].append({
                                            "properties": {"name": 50*i + j, 
                                                           "style": {"color": "white", "weight": 0.5, 
                                                                     'fillColor':"red", "fillOpacity": 0.2}},
                                            "id": 50*i + j,
                                            "type": "Feature",
                                            "geometry": {
                                                        "type": "Polygon",
                                                        "coordinates": [[[long_grid[i], lat_grid[j]],
                                                                         [long_grid[i], lat_grid[j+1]], 
                                                                         [long_grid[i+1], lat_grid[j+1]],
                                                                         [long_grid[i+1], lat_grid[j],
                                                                       ]]]
                                                }
                                           })       

In [10]:
create_reg_grid(ny_json)

##### styledict = { <br> &nbsp;&nbsp;&nbsp;&nbsp; '0': { '2017-1-1': {'color': 'ffffff', 'opacity': 1} <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; '2017-1-2': {'color': 'fffff0', 'opacity': 1} <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;... <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;}, <br>  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;..., <br> &nbsp;&nbsp;&nbsp;&nbsp; 'n': { <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; '2017-1-1': {'color': 'ffffff', 'opacity': 1} <br>  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; '2017-1-2': {'color': 'fffff0', 'opacity': 1} <br>  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ... <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  } <br>  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }

In [11]:
styledict_actual = {}
styledict_predicted = {}

max_val_act = np.sqrt(verify_taxi.values.max())
max_val_pred = np.sqrt(data_predict.fillna(0).values.max())

def create_style_grid(style_json, df, max_val):
    for region in range(1, 2501):
        d = {}
        if region in df.columns.values:
            for t in df.index.values[744:816]: 
                ''' first three days of june 2016 '''
                time = ((t - np.datetime64('1970-01-01T03:00:00')) / np.timedelta64(1, 's')).astype(str)
                d[time] = {'color': 'red', 'opacity': np.sqrt(df.loc[t, region]) / max_val}
            style_json[region] = d    

In [12]:
create_style_grid(styledict_actual, verify_taxi, max_val_act)
create_style_grid(styledict_predicted, data_predict, max_val_pred)

In [17]:
print(len(ny_json["features"]), len(styledict_actual))

##### to reduce the spread of the taxi calls РјС„РґРіСѓ - a square root was taken. Without this, the visualization map plotted by intensity of color (to be more precise - the degree of transparency) would be predominantly white.

##### in class TimeSliderChoropleth folium package I have changed method toDateString() on  toString() - in the part of code written in JavaScript and relats to the time slider visualization. Without this change time was showed without its hours part. Only year, month and day.

In [13]:
width, height = 980, 940

fig = Figure(width=width, height=height, ratio="60%")
subplots, f_map, t_chor  = [], [], []

for idx, s in enumerate([styledict_actual, styledict_predicted]):
    subplots.append(fig.add_subplot(1, 2, idx+1, margin=0.07))

    f_map.append(folium.Map([40.720610, -73.905242], tiles='cartodbpositron', zoom_start=11, width= '100%', height = '100%'))#width= 470, height = 520)
    t_chor.append(TimeSliderChoropleth(
                                       ny_json,
                                       styledict=s,).add_to(f_map[idx]))

    folium.TileLayer('openstreetmap').add_to(f_map[idx])
    folium.TileLayer('stamenterrain').add_to(f_map[idx])
    folium.TileLayer('Mapbox Bright').add_to(f_map[idx])
    folium.LayerControl().add_to(f_map[idx])

    subplots[idx].add_child(f_map[idx])

fig 

##### 2) Second version

In [14]:
ny_json_actual = {
            "type": "FeatureCollection",
            "features": []
          } 

ny_json_predicted = {
            "type": "FeatureCollection",
            "features": []
          } 

In [15]:
create_reg_grid(ny_json_actual)
create_reg_grid(ny_json_predicted)

In [16]:
def plot_dict(reg_num):
    #print(reg_num)
    end_time = verify_taxi.loc[:, reg_num].index.values[336]
    if reg_num in verify_taxi.columns.values:
        actual_df = verify_taxi.loc[:end_time, reg_num]
        predict_df = data_predict.loc[:end_time, reg_num]

        trace_actual = go.Scatter(
            x=list(map(str,actual_df.index.values)),
            y=actual_df.values,
            name = "actual",
            line = dict(color = '#17BECF'),
            opacity = 0.8)

        trace_prediction = go.Scatter(
            x=list(map(str,predict_df.index.values)),
            y=predict_df.values,
            name = "prediction",
            line = dict(color = '#ffa500'),
            opacity = 0.8)

        data = [trace_actual, trace_prediction]

        layout = dict(
            title='Taxi Time Series',
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                             label='1d',
                             step='day',
                             stepmode='backward'),
                        dict(count=7,
                             label='7d',
                             step='day',
                             stepmode='backward'),
                        dict(step='all')
                    ])
                ),
                rangeslider=dict(),
                type='date'
            )
        )

        fig = dict(data=data, layout=layout)
        
        return fig, "Taxi Time Series"       

In [17]:
def plot_pediction(figure, name):
    display(py.iplot(figure, filename = name))

In [18]:
def f(x):
    ''' .OpenStreetMap.Mapnik OpenStreetMap.BlackAndWhite, OpenMapSurfer.Grayscale, Esri.WorldStreetMap, Hydda.Full'''
    m, label, region = [], [], []
    
    def handle_interaction(**kwargs):
        if kwargs.get('type') == 'mousedown':
            coords = kwargs.get('coordinates')
            long, lat = (coords[1] - NY_long_west), (coords[0] - NY_latt_south)
            long_size, lat_size = abs(NY_long_east - NY_long_west), abs(NY_latt_north - NY_latt_south)
            r = (long // (long_size / 50)) * 50 +  (lat // (lat_size / 50))
            if int(r) in verify_taxi.columns.values:
                region[idx].value = str(int(r))
                figure, plot_name = plot_dict(reg_num=int(r))
                display(py.iplot(figure, filename = plot_name))
                #plot_pediction(figure, plot_name)
                
        if kwargs.get('type') == 'mousemove':
            label[idx].value = str(kwargs.get('coordinates'))
    
    for idx, ny_json in enumerate([ny_json_actual, ny_json_predicted]):
        m.append(Map(center=[40.720610, -73.905242], zoom=11, basemap= basemaps.Hydda.Full, 
                     layout=dict(width='480px', height='600px')))
        
        for layer in [basemaps.OpenMapSurfer.Roads, basemaps.Esri.WorldStreetMap, basemaps.OpenMapSurfer.Grayscale, 
                      basemaps.Esri.WorldImagery, basemaps.OpenStreetMap.France,]:
            m[idx].add_layer(basemap_to_tiles(layer))

        m[idx].add_layer(GeoJSON(data=ny_json))
        m[idx].add_layer(GeoJSON(data=ny_json, hover_style={'fillColor': "#4292c6", 'fillOpacity': 0.7}, name='regions'))

        # Adding the control
        m[idx].add_control(LayersControl())

        label.append(Label())
        region.append(Label())
        
        m[idx].on_interaction(handle_interaction)

        display(label[idx])
        display(region[idx])

    
    box_layout = widgets.Layout(display='flex', flex_flow='column', width='50%', height='600px')
    
    map_1 = widgets.VBox([widgets.Text('actual', disabled=True), m[0]], layout=box_layout)
    map_2 = widgets.VBox([widgets.Text('prediction', disabled=True), m[1]], layout=box_layout)
    
    maps= widgets.HBox(children=[map_1, map_2])
    display(maps)

##### left click on any of 102 regions - and a graph with real and predicted values for this region will appear under the NY map plot. its possible to change map layouts - thus giving the map a different appearance

In [19]:
max_actual = np.sqrt(verify_taxi.values.max())
max_predict = np.sqrt(data_predict.fillna(0).values.max())

def update(change):
        time = hour_slider.value
        #print(time)
        for idx, reg in enumerate(verify_taxi.columns.values):
            if ny_json_actual["features"][idx]["id"] == reg: 
                ny_json_actual["features"][idx]["properties"]["style"]["fillOpacity"] =\
                                                                np.sqrt(verify_taxi.loc[time, reg]) / max_actual
            if ny_json_predicted["features"][idx]["id"] == reg:        
                ny_json_predicted["features"][idx]["properties"]["style"]["fillOpacity"] =\
                                                                np.sqrt(data_predict.loc[time, reg]) / max_predict 
                

hour_slider = widgets.SelectionSlider(description="time", options=list(verify_taxi.index.values[744:816]),
                                           continuous_update=False, value=verify_taxi.index.values[744])  
                    
hour_slider.observe(update, names="value") 

interact(f, x=hour_slider); 

In [ ]:
'''
basemaps.OpenStreetMap.Mapnik, basemaps.OpenStreetMap.BlackAndWhite, basemaps.OpenStreetMap.France, basemaps.OpenMapSurfer.Roads, basemaps.Esri.WorldStreetMap,basemaps.Esri.WorldImagery, basemaps.Esri.NatGeoWorldMap
basemaps.OpenStreetMap.BlackAndWhite, basemaps.OpenMapSurfer.Grayscale

basemaps.OpenStreetMap.HOT, , basemaps.OpenStreetMap.DE
basemaps.OpenMapSurfer.Grayscale
basemaps.Esri.WorldTopoMap 
'''